# Interaktive Python-Beispiele zu Supervised, Unsupervised und Reinforcement Learning

Wir schauen uns drei grundlegende Arten von maschinellem Lernen an:

1. **Supervised Learning** – Lernen mit Lösungen (Beispiele mit richtiger Antwort)
2. **Unsupervised Learning** – Muster finden ohne Lösungen
3. **Reinforcement Learning** – Lernen durch Belohnung und Ausprobieren

Zu jedem Typ gibt es **eine kurze Erklärung** und **ein kleines interaktives Beispiel**.


In [1]:
# Gemeinsame Bibliotheken
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ipywidgets as widgets
from ipywidgets import interact

---
## 1️. Supervised Learning – Lernen mit Lösungen

Beim **Supervised Learning** bekommt das Modell **Beispiele mit richtiger Lösung**.

Beispiel-Idee:
- Eingabe: zwei Zahlen (x und y)
- Ausgabe: Klasse **A** oder **B** (z. B. rote oder blaue Punkte)

Das Modell soll lernen, **eine Grenze** zwischen den Klassen zu finden.

Im folgenden Beispiel:
- Die **wahre Klasse** (A oder B) ist im Hintergrund definiert.
- Mit einer **Geraden** sollen Sie versuchen, die Klassen zu trennen.
- Die Punkte werden **nach Vorhersage eingefärbt**:
  - rot = Modell sagt Klasse A
  - blau = Modell sagt Klasse B
- Falsch klassifizierte Punkte werden mit einem **schwarzen Rand** markiert.


In [2]:
# Datenset für Klassifikation erzeugen (2 Klassen)
np.random.seed(0)
n_points = 40

# Klasse A (0) – um (1,1)
x1_a = np.random.normal(loc=1.0, scale=0.5, size=(n_points//2, 2))
# Klasse B (1) – um (3,3)
x1_b = np.random.normal(loc=3.0, scale=0.5, size=(n_points//2, 2))

X_sup = np.vstack([x1_a, x1_b])
y_true = np.array([0]*(n_points//2) + [1]*(n_points//2))  # 0 = A, 1 = B

def plot_supervised(m=1.0, b=0.0):
    """
    Interaktives Modell: Klassifikation mit einer verschiebbaren Geraden.
    - Rot / Blau: vom Modell vorhergesagte Klasse
    - Schwarzer Rand: falsch klassifizierte Punkte
    """
    # Neue Figur für jeden Aufruf
    plt.figure(figsize=(5, 5))

    # Gerade: y = m*x + b
    xs = np.linspace(0, 4.5, 100)
    ys = m * xs + b

    # Vorhersage des "Modells": oberhalb Linie -> Klasse 1 (blau), sonst Klasse 0 (rot)
    y_pred = (X_sup[:, 1] > (m * X_sup[:, 0] + b)).astype(int)

    # Farben NACH Vorhersage
    colors = np.where(y_pred == 0, "red", "blue")

    # Falsch klassifizierte Punkte markieren
    wrong = y_pred != y_true

    # Korrekt klassifizierte Punkte
    plt.scatter(
        X_sup[~wrong, 0],
        X_sup[~wrong, 1],
        c=colors[~wrong],
        label="korrekt klassifiziert",
    )

    # Falsch klassifizierte Punkte mit schwarzem Rand
    if wrong.any():
        plt.scatter(
            X_sup[wrong, 0],
            X_sup[wrong, 1],
            c=colors[wrong],
            edgecolors="black",
            linewidths=1.5,
            label="falsch klassifiziert",
        )

    # Entscheidungsgrenze zeichnen
    plt.plot(xs, ys, "k-", label=f"Gerade: y = {m:.2f} * x + {b:.2f}")

    # Accuracy berechnen
    acc = (y_pred == y_true).mean()

    plt.xlim(0, 4.5)
    plt.ylim(0, 4.5)
    plt.xlabel("Merkmal 1")
    plt.ylabel("Merkmal 2")
    plt.title("Supervised Learning – eigene Entscheidungsgrenze")
    plt.grid(True)
    plt.legend()
    plt.show()

    print(f"Anteil korrekt klassifizierter Punkte: {acc * 100:.1f} %")

interact(
    plot_supervised,
    m=widgets.FloatSlider(
        value=1.0, min=-2.0, max=2.0, step=0.1, description="Steigung m"
    ),
    b=widgets.FloatSlider(
        value=0.0, min=-2.0, max=4.0, step=0.1, description="Achsenabschnitt b"
    ),
);

interactive(children=(FloatSlider(value=1.0, description='Steigung m', max=2.0, min=-2.0), FloatSlider(value=0…

**Aufgabe:**  
Spielen Sie mit der Steigung **m** und dem Achsenabschnitt **b**, bis die Gerade die beiden Klassen möglichst gut trennt.

- Schafft Sie mehr als **90 %** korrekt klassifizierte Punkte?
- Was fällt auf, wenn die Klassen sich überlappen?


---
## 2️. Unsupervised Learning – Muster ohne Lösungen

Beim **Unsupervised Learning** gibt es **keine vorgegebenen Klassen** oder richtigen Antworten.

Das Modell versucht selbst, **Muster oder Gruppen** in den Daten zu finden.

Hier nutzen wir **K-Means-Clustering**:
- Sie können auswählen, **wie viele Gruppen (Cluster)** gesucht werden sollen.
- Das Modell färbt die Punkte entsprechend ein.


In [3]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

# Daten für Clustering erzeugen
X_unsup, _ = make_blobs(n_samples=60, centers=3, cluster_std=0.6, random_state=42)

def plot_unsupervised(k=3):
    # K-Means mit k Clustern
    km = KMeans(n_clusters=k, n_init=10, random_state=0)
    km.fit(X_unsup)
    labels = km.labels_
    centers = km.cluster_centers_

    plt.figure(figsize=(5,5))
    plt.scatter(X_unsup[:,0], X_unsup[:,1], c=labels, cmap="tab10", s=40)
    plt.scatter(centers[:,0], centers[:,1], c="black", marker="X", s=100, label="Cluster-Zentren")
    plt.title(f"Unsupervised Learning – K-Means mit k = {k} Clustern")
    plt.xlabel("Merkmal 1")
    plt.ylabel("Merkmal 2")
    plt.grid(True)
    plt.legend()
    plt.show()

interact(plot_unsupervised,
         k=widgets.IntSlider(value=3, min=1, max=6, step=1, description="Anzahl Cluster k"));

interactive(children=(IntSlider(value=3, description='Anzahl Cluster k', max=6, min=1), Output()), _dom_classe…

**Beobachtungsfragen:**  

1. Wie sieht das Ergebnis aus, wenn **k = 1** definiert ist?  
2. Was passiert, wenn man **mehr Cluster wählt, als es „natürliche“ Gruppen gibt**?  
3. Inwiefern ist das anders als beim Supervised-Learning-Beispiel oben?


---
## 3️. Reinforcement Learning – Lernen durch Belohnung

Ein einfaches Beispiel für **Reinforcement Learning (RL)** ist der sogenannte **Multi-Armed Bandit**.

Stell Sie sich vor, es gibt **3 Spielautomaten** (Arme):

- Jeder Automat gibt mit einer bestimmten Wahrscheinlichkeit Punkte.
- Diese Wahrscheinlichkeiten sind aber **nicht** bekannt.
- Sie müssen ausprobieren (explorieren) und dann die besten nutzen (exploiten).

Das Ziel: **Insgesamt möglichst viele Punkte sammeln.**


### Der Lern-Algorithmus (ε-greedy)

Ein Agent macht nun bei jedem Zug Folgendes:

1. Mit Wahrscheinlichkeit **ε**:  
   -> Er wählt einen **zufälligen** Automaten (Exploration).

2. Mit Wahrscheinlichkeit **1 − ε**:  
   -> Er wählt den Automaten mit der **bisher höchsten geschätzten Belohnung** (Exploitation).

Nach jedem Zug:
- bekommt er eine Belohnung (0 oder 1)
- aktualisiert seine **Schätzung** für diesen Automaten.


In [4]:
# Echte (versteckte) Gewinnwahrscheinlichkeiten der 3 Automaten
true_probs = np.array([0.2, 0.5, 0.8])

def run_bandit(epsilon=0.2, steps=100):
    # geschätzte Belohnung pro Automat (startet bei 0)
    est_values = np.zeros(3)
    # wie oft wurde jeder Automat gewählt?
    counts = np.zeros(3)
    # Liste der erhaltenen Belohnungen
    rewards = []

    for t in range(steps):
        # Entscheidung: explore oder exploit?
        if np.random.rand() < epsilon:
            # Exploration: zufälliger Automat
            action = np.random.randint(0, 3)
        else:
            # Exploitation: Automat mit bisher höchstem geschätzten Wert
            action = np.argmax(est_values)

        # Belohnung simulieren: 1 mit Wahrscheinlichkeit true_probs[action], sonst 0
        reward = 1 if np.random.rand() < true_probs[action] else 0

        # Schätzung aktualisieren (einfacher gleitender Durchschnitt)
        counts[action] += 1
        est_values[action] += (reward - est_values[action]) / counts[action]

        rewards.append(reward)

    # Durchschnittliche Belohnung pro Schritt
    avg_rewards = np.cumsum(rewards) / (np.arange(steps) + 1)

    # Plot: durchschnittliche Belohnung im Zeitverlauf
    plt.figure(figsize=(6,4))
    plt.plot(avg_rewards)
    plt.xlabel("Schritt")
    plt.ylabel("Durchschnittliche Belohnung")
    plt.title("Durchschnittliche Belohnung über die Zeit")
    plt.grid(True)
    plt.show()

    print("Echte Gewinnwahrscheinlichkeiten:   ", true_probs)
    print("Geschätzte Werte nach dem Lernen:  ", np.round(est_values, 2))
    print("Anzahl Züge pro Automat:           ", counts.astype(int))

interact(
    run_bandit,
    epsilon=widgets.FloatSlider(
        value=0.2, min=0.0, max=1.0, step=0.05, description="Exploration ε"
    ),
    steps=widgets.IntSlider(
        value=100, min=20, max=500, step=20, description="Schritte"
    ),
);

interactive(children=(FloatSlider(value=0.2, description='Exploration ε', max=1.0, step=0.05), IntSlider(value…

**Experimentierfragen:**  

Probieren Sie Folgendes aus:

1. Setzen Sie **ε = 0.0**  
   - Was passiert, wenn der Agent **nie exploriert**?
   - Findet er immer den besten Automaten?

2. Setzt Sie **ε = 0.9**  
   - Was passiert, wenn der Agent **fast immer zufällig** spielt?
   - Wird die durchschnittliche Belohnung gut oder schlecht?

3. Ausprobieren von Anzahl **Schritte**.  
   - Wie nah kommen die **geschätzten Werte** an die **echten** heran?
   - Wieviele Schritte werden dafür benötigt?
   - Welcher Automat wird am häufigsten gespielt? Passt das zu den echten Wahrscheinlichkeiten?


---
## Zusammenfassung

In diesem Notebook wurden drei grundlegende Arten von Machine Learning kennengelernt:

1. **Supervised Learning:**  
   - Lernen mit Eingaben **und** vorgegebenen richtigen Ausgaben  
   - Beispiel: Klassifikation von Punkten (rot/blau) mit einer Entscheidungsgrenze

2. **Unsupervised Learning:**  
   - Lernen **ohne** vorgegebene richtige Antworten  
   - Beispiel: K-Means-Clustering, das Gruppen in Daten findet

3. **Reinforcement Learning:**  
   - Lernen durch **Belohnung und Ausprobieren**  
   - Beispiel: Ein Agent probiert verschiedene „Spielautomaten“ aus und lernt, welche am meisten Punkte bringen.

